In [7]:
import salabim_plus as sim_plus
import pandas as pd
import salabim as sim
import multiplexer
import OPTX
import BFPD
import splitter
import horn_antenna
import section_A
import section_B
import section_C
import section_D
import section_E
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 100

with open('output_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
    env = sim_plus.EnvironmentPlus(trace=out)
    
    # step 2: create the machines in the simulation environment
    # do there need to be things like assembly benches here?
    VSWR_CTI1         =sim_plus.Machine(var_name='VSWR_CTI1',        env=env)
    VSWR_CTI_DBG      =sim_plus.Machine(var_name='VSWR_CTI_DBG',     env=env)
    OPTX_CTI          =sim_plus.Machine(var_name='OPTX_CTI',         env=env)
    BFPD_CTI          =sim_plus.Machine(var_name='BFPD_CTI',         env=env)
    BFPD_CTI_DBG      =sim_plus.Machine(var_name='BFPD_CTI_DBG',     env=env)
    VSWR_CTI2         =sim_plus.Machine(var_name='VSWR_CTI2',        env=env)
    # maybe make VSWR_CTI1 and VSWR_CTI2 into a machine group that 
    # goes by the name of VSWR_CTI
    MOD_FEEDASSY_PAT  =sim_plus.Machine(var_name='MOD_FEEDASSY_PAT', env=env)
    static_CTI        =sim_plus.Machine(var_name='STATIC_CTI',       env=env)
    static_CTI_DBG    =sim_plus.Machine(var_name='STATIC_CTI_DBG',   env=env)
    COND_EST          =sim_plus.Machine(var_name='COND_EST',         env=env)
    RF_Fit_Test       =sim_plus.Machine(var_name='RF_FIT_TEST',      env=env)
    RF_CFG            =sim_plus.Machine(var_name='RF_CFG',           env=env)
 
    assembly_bench_1  = sim_plus.Machine(var_name='assembly_bench_1', env=env)
    #how many assembly benches should we create?
    
    quality_bench_1   = sim_plus.Machine(var_name='quality_bench_1',  env=env)
    #how many quality benches should we create?

    # step 3: make machine grouping for common processes
    assembly_bench = sim_plus.MachineGroup(var_name='assembly_bench', env=env,
                                           machines=[assembly_bench_1])
    quality_bench  = sim_plus.MachineGroup(var_name='quality_bench', env=env,
                                           machines=[quality_bench_1])
    

    
    # step 4: create the workers in the simulation environment
    # how many workers actually are there??
    assembler                  =sim_plus.Worker(var_name='assembler',    env=env, capacity=1)
    technician                 =sim_plus.Worker(var_name='technician',   env=env, capacity=1)
    production_control         =sim_plus.Worker(var_name='production_control', env=env, capacity=1)
    qual_inspector             =sim_plus.Worker(var_name='inspection',   env=env, capacity=1)
    
    
    # step 5: make the shift schedules for the workers
    
    # these probably need to be checked for correctness,
    # not sure if start_time=0 or if shift_type=pattern
    
    shift_schedule = misc_tools.make_shifts(shift_duration=24*60, 
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    production_control_shift = sim_plus.ShiftController(worker=production_control,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    qual_inspector_shift = sim_plus.ShiftController(worker=qual_inspector,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    # step 6: point to the step creation function
    
    # if we pull data from the database, can we update the create_routing and pass in an argument of the 
    # arrays that we want them to pull from / reference?
    
    # call create_routing for the components here
    
    # step 7: make all of the generators that determine when entites are made
    # call sim_plus.EntityGenerator for each of the componenets
    
    # step 8: create all of the inventory locations in the simulation environment
    # call sim_plus.Kanban for each of the components with inventory systems
    
    # step 9: connect any entity boms and main_exits to its entity generator
    # e.g. part_c_gen.bom = part_c.get_bom(env=env.objs)
    #      part_a.main_exit = part_a_kanban
    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    production_control_shift.activate(process='work')
    qual_inspector_shift.activate(process='work')
    
    # activate components
    # e.g. part_a_gen.activate(process='arrive')
    
    # stepp 11: execute the simulation
    env.run(till=runtime)

<class 'module'>
